In [1]:
%config Completer.use_jedi = False

import psycopg2
import pandas as pd
import numpy as np
from dotenv import dotenv_values
from sqlalchemy import create_engine
import requests
import time
import json

### Connection to AWS

In [3]:
# get the connection details for the rds db from .env file
config = dotenv_values(".env")  
HOST_RDS = config['HOST_RDS']
DBNAME_RDS = config['DBNAME_RDS']
USER_RDS = config['USER_RDS']
PASSWORD_RDS = config['PASSWORD_RDS']

In [47]:
# create df from postgreSQL tables
engine_str = 'postgresql+psycopg2://' + USER_RDS + ':' + PASSWORD_RDS + "@" + HOST_RDS + ":5432/dbeq"
engine = create_engine(engine_str)
dbConnection = engine.connect()
twitter = pd.read_sql("SELECT * FROM dl_twitter", dbConnection)
users = pd.read_sql("SELECT * from dl_twitterusers", dbConnection)

In [31]:
print(twitter[twitter.duplicated(subset=['id'])])
print(users[users.duplicated(subset=['id'])])

Empty DataFrame
Columns: [text, author_id, id, created_at]
Index: []
                          id         username                          name  \
11                       246           blaine               Blaine Cook 💉💉🎉   
823                   615403       avalonstar  Bryan Veloso 🌟 Forging Ahead   
873                   630163          superic                 Eric Willis 🦃   
886                   634703            k3nnr                     k3nnr.ogg   
889                   635463    FauziKHamadeh              Fauzi K. Hamadeh   
...                      ...              ...                           ...   
2501742  1406414300311212034   yourlight51129      🌟🌙 🥚let's shine together   
2501794  1406530652841852934        NewsBits_                     News Bits   
2502333  1407964313793425411  viral48official                       viral48   
2505737  1417005186262867973        MNSTAPLZA                   I MISS BTS.   
2507964  1422990264147185666  NewsFromBehind1              New

In [14]:
# removing duplicates (users where extracted multiple times because of 500 tweets per request limit)
users.drop_duplicates(subset=['id'], inplace=True)
print(users[users.duplicated(subset=['id'])])

Empty DataFrame
Columns: [id, username, name, location]
Index: []


In [15]:
# combine tables
df = pd.merge(twitter, users, how='left', left_on='author_id', right_on='id')
df = df.rename(columns={"id_x": "tweet_id"}).drop(columns=['id_y'])
# drop column id from users side, as it is the same as author_id from twitter table.


In [73]:
df.dtypes

text          object
author_id      int64
tweet_id       int64
created_at    object
username      object
name          object
location      object
dtype: object

In [16]:
# change created_at dtype
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_localize(None)

In [38]:
# some users could not be inserted into postgreSQL table due to invalid characters. We identify them:
df[df['username'].isna()]


,text,author_id,tweet_id,created_at,username,name
4,ｗｈｅｎ ｉ ｆｅｅｌ ｅａｒｔｈｑｕａｋｅ， ｍｙ ｈａｎｄ ａｕｔｏｍａｔｉｃａｌｌｙ ...,2760893045,884990764933148673,2017-07-12 04:20:15,NaN,NaN
5,ｗｈｅｎ ｉ ｆｅｅｌ ｅａｒｔｈｑｕａｋｅ， ｍｙ ｈａｎｄ ａｕｔｏｍａｔｉｃａｌｌｙ ...,3214732451,921250488313622528,2017-10-20 05:43:27,NaN,NaN
23,ｗｈｅｎ ｉ ｆｅｅｌ ｅａｒｔｈｑｕａｋｅ， ｍｙ ｈａｎｄ ａｕｔｏｍａｔｉｃａｌｌｙ ...,36488329,928040346604523520,2017-11-07 23:23:55,NaN,NaN
30,ｗｈｅｎ ｉ ｆｅｅｌ ｅａｒｔｈｑｕａｋｅ， ｍｙ ｈａｎｄ ａｕｔｏｍａｔｉｃａｌｌｙ ...,308947696,928030577537589248,2017-11-07 22:45:06,NaN,NaN
49,ｗｈｅｎ ｉ ｆｅｅｌ ｅａｒｔｈｑｕａｋｅ， ｍｙ ｈａｎｄ ａｕｔｏｍａｔｉｃａｌｌｙ ...,3062164543,938577830752935936,2017-12-07 01:16:07,NaN,NaN
...,...,...,...,...,...,...
13517310,鹿児島で震度5強。：https://t.co/EKgCD3bD0Q鹿児島で震度5強。またです...,266278707,885140873863872512,2017-07-12 14:16:44,NaN,NaN
13517311,鹿児島 震度5強\n\nhttps://t.co/vm5PTMfJuh https://t....,875992792773738496,884609069700403200,2017-07-11 03:03:32,NaN,NaN
13517313,鹿児島で震度5強\n\n九州は散々やで…(*_*)\n\n地震情報 - Yahoo!天気・災...,86334263,884609347925426177,2017-07-11 03:04:38,NaN,NaN
13517315,鹿児島で震度5強の地震\nhttps://t.co/Gl9wT2IQz4,2414138750,884609357027041281,2017-07-11 03:04:40,NaN,NaN


In [7]:
# we could either delete those tweets or try to insert those users. Since we have access to the data, we will insert it. However, getting the users by author_id would take too long, as there are 
# 240239 unique users missing. We will rather get the tweets with the users again for that timeframe, remove the invalid character and insert it into the table.
missing = df[df.isna().any(axis=1)].copy()
missing['created_at'] = pd.to_datetime(missing['created_at']).dt.date
missing.sort_values(by='created_at', inplace=True)

uniqueDates = missing['created_at'].unique()
print(uniqueDates)
# from the output we see the affected time periods are: 1.7.17-31.12.17, 1.7.18-13.10.18 and then one day on 9.7.21 (these we will delete, they are only a few tweets and won't change the analysis)

[datetime.date(2017, 7, 1) datetime.date(2017, 7, 2)
 datetime.date(2017, 7, 3) datetime.date(2017, 7, 4)
 datetime.date(2017, 7, 5) datetime.date(2017, 7, 6)
 datetime.date(2017, 7, 7) datetime.date(2017, 7, 8)
 datetime.date(2017, 7, 9) datetime.date(2017, 7, 10)
 datetime.date(2017, 7, 11) datetime.date(2017, 7, 12)
 datetime.date(2017, 7, 13) datetime.date(2017, 7, 14)
 datetime.date(2017, 7, 15) datetime.date(2017, 7, 16)
 datetime.date(2017, 7, 17) datetime.date(2017, 7, 18)
 datetime.date(2017, 7, 19) datetime.date(2017, 7, 20)
 datetime.date(2017, 7, 21) datetime.date(2017, 7, 22)
 datetime.date(2017, 7, 23) datetime.date(2017, 7, 24)
 datetime.date(2017, 7, 25) datetime.date(2017, 7, 26)
 datetime.date(2017, 7, 27) datetime.date(2017, 7, 28)
 datetime.date(2017, 7, 29) datetime.date(2017, 7, 30)
 datetime.date(2017, 7, 31) datetime.date(2017, 8, 1)
 datetime.date(2017, 8, 2) datetime.date(2017, 8, 3)
 datetime.date(2017, 8, 4) datetime.date(2017, 8, 5)
 datetime.date(2017, 8, 

### Extract additional data

In [15]:
BEARER_TOKEN = config['BEARER_TOKEN']
FILTER_QUERY = "-from:quakeupdates -from:jojo2727 -from:MonitorSismico -from:MyComicalLife -from:news_sokuho_bot -from:DiariosRobot -from:EN_NERV -from:GDACS -from:earthquake_jp -from:EQAlerts -from:j1_quake -from:iSachinSrivstva -from:VolcanoEWS -from:ChileAlertaApp -from:earthb0t -from:sexy_vegetables -from:zishin3255 -from:everyEarthquake -from:MapQuake -from:swap_bot_bash -from:eq_map -from:eq_map_es -from:eq_map_ww -from:SEISMOinfo -from:VegaBajaWx -from:WatchOurCity -from:Keith_Event -from:SismoDetector -from:cvb_223 -from:ExBulletinUk -from:EMSC -from:StoixeioJewelry -from:megamodo -from:earthquakevt -from:QuakeBotter -from:twtaka_jp -from:EarthquakeTw -from:ENSO1998 -from:eq_map_ww2 -from:eq_map_es2"


start_time = '2017-07-01T00:00:00.000Z'
end_time = "2017-12-31T23:59:59.000Z"
query = "earthquake -minor, -is:reply -is:retweet {0}".format(FILTER_QUERY)
max_results = "500"
tweet_fields = "created_at,author_id"
user_fields = 'username,location'
expansions = 'author_id'
counter = 0
query_params = {'query': query, 'tweet.fields': tweet_fields, 'user.fields': user_fields, \
                'start_time': start_time, 'end_time': end_time, 'max_results': max_results, \
                'expansions': expansions}
url = "https://api.twitter.com/2/tweets/search/all"
headers = {"Authorization": "Bearer " + BEARER_TOKEN}

In [16]:
lst_tweets = []
lst_users = []
while True:
    # get results according to url and query
    response = requests.request("GET", url, headers=headers, params=query_params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

    # combine data to one
    json_response = response.json()
    if 'data' in json_response:
        lst_tweets = lst_tweets + json_response['data']
        lst_users = lst_users + json_response['includes']['users']

    # check if more data available, if yes continue process
    if 'meta' in json_response:
        if 'next_token' in json_response['meta']:
            query_params['next_token'] = json_response['meta']['next_token']
            next_token = json_response['meta']['next_token']
          #  logging.info("Fetching next few tweets, next_token: ", query_params['next_token'])
            time.sleep(5)
        else:
            if 'next_token' in query_params:
                del query_params['next_token']
            break
    else:
        if 'next_token' in query_params:
            del query_params['next_token']
            print('no meta in json_response')
        break

In [ ]:
df_t = pd.DataFrame(lst_tweets)
df_u = pd.DataFrame(lst_users)



In [ ]:
# because of memory issues the data was saved to csv and will be read in here again
twe = pd.read_csv("matchingtweets.csv")
use = pd.read_csv("missingusers.csv")

df2 = pd.merge(twe, use, how='left', left_on='author_id', right_on='id')
df2 = df2.rename(columns={"id_x": "tweet_id"}).drop(columns=['id_y'])

# delete location because of no relevant information
df2.drop(columns=['location'], inplace=True)
df.drop(columns=['location'], inplace=True)

df2['created_at'] = pd.to_datetime(df2['created_at']).dt.tz_localize(None)

df3 = df2.append(df)
df3.drop_duplicates(subset=['tweet_id'], inplace=True)


In [ ]:
# check for nan again, on column username
df3[df3['username'].isna()] # significantly less. 

,tweet_id,created_at,text,author_id,username,name
776569,911360842528100352,2017-09-22 22:45:31,"The reason Frida, the trapped little girl, gri...",186552155,NaN,jeff
26472,1059683238170542080,2018-11-06 05:45:46,02/14/14: I’m watching House of Cards (S2) in ...,156334774,NaN,NaN
54200,1056912735244562434,2018-10-29 14:16:47,"0.4, 0.8, 1.1 \nIf this appears on #OnlyConnec...",310455504,NaN,NaN
64102,1054783429970137088,2018-10-23 17:15:41,0.4 level earthquake today @NoFrackLancs Time...,4331730273,NaN,NaN
106104,1052907078615031808,2018-10-18 12:59:44,074 // earthquake // #100daysofascent #100dayc...,23904102,NaN,NaN
...,...,...,...,...,...,...
13514569,1052127631221514241,2018-10-16 09:22:29,「ドーン！」と音がしただけで何かよくわからなかったけど、最近多いらしい…\n大地震の前兆とか...,550738293,NaN,NaN
13514660,1061534708427706369,2018-11-11 08:22:51,この頃、また地震が多い…。\n地震情報 - Yahoo!天気・災害 https://t.co...,110049763,NaN,NaN
13515815,1058274420068741120,2018-11-02 08:27:38,カラスが騒がしかったのと、家鳴りがあったのはこれかな？\n地震情報 - Yahoo!天気・災...,2966046025,NaN,NaN
13516249,1058266760590254080,2018-11-02 07:57:12,高知市内揺れたね、和歌山の方が震源みたい。#地震 #和歌山 #震度4 https://t.c...,198874833,NaN,NaN


In [ ]:
df3.to_csv('temp_df3.csv', index=False)

## Remove Bots

In [4]:
df = pd.read_csv("temp_df3.csv")

In [13]:
# start with removing all users where username contains 'bot' or 'quake'
remove = ['quake', 'bot']
dw = df[~df.username.str.contains('|'.join(remove), case=False, na=False)]


In [36]:
# remove users which posted more tweets than certain threshhold
post_counts = dw.username.value_counts()
lst_user = post_counts.index.tolist()
lst_count = post_counts.tolist()

post_counts = pd.DataFrame(lst_user).rename(columns={0: 'username'})
post_counts['value'] = lst_count


bots = post_counts.loc[post_counts['value'] > 100].sort_values(by=['value'])
bots = bots.username.tolist()

#dw[~dw.username.str.contains('|'.join(bots), case=False, na=False)]

In [39]:
dw2 = dw[~dw.username.str.contains('|'.join(bots), case=False, na=False)]

In [41]:
twitter.to_sql('twitter', con=engine, if_exists='append', chunksize=1000, index=False)


In [48]:
dw.to_sql('dw_twitter', con=engine, if_exists='append', chunksize=1000, index=False)

            

In [ ]:
# filter out accounts with more or equal to 175 postings
df = df.loc[df['value'] > 10].sort_values(by=['value'])
lst_bots = df_a.username.tolist()



# add -from: operator to each entry in list 
append_str = '-from:'
lst_filter = [append_str + sub for sub in lst_bots]

# convert list to a string suitable for the query
filter_names_query = ' '.join(lst_filter)
